In [2]:
import os
import sys
import urllib, io
os.getcwd()
sys.path.append("..")
sys.path.append("../../utils")
sys.path.append("../../analysis/utils")


import numpy as np
import scipy.stats as stats
import pandas as pd
from sklearn.cluster import AffinityPropagation
from sklearn.manifold import TSNE

import pymongo as pm
from collections import Counter
import json
import re
import ast

from PIL import Image, ImageOps, ImageDraw, ImageFont 

from io import BytesIO
import base64

import  matplotlib
from matplotlib import pylab, mlab, pyplot
%matplotlib inline
from IPython.core.pylabtools import figsize, getfigs
plt = pyplot
import matplotlib as mpl
mpl.rcParams['pdf.fonttype'] = 42

import seaborn as sns
sns.set_context('talk')
sns.set_style('darkgrid')

from IPython.display import clear_output

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", message="numpy.dtype size changed")
warnings.filterwarnings("ignore", message="numpy.ufunc size changed")

# import drawing_utils as drawing
import importlib
import scoring

In [3]:
## directory & file hierarchy
proj_dir = os.path.abspath('../..')
datavol_dir = os.path.join(proj_dir,'data')
analysis_dir =  os.path.abspath('.')
results_dir = os.path.join(proj_dir,'results')
plot_dir = os.path.join(results_dir,'plots')
csv_dir = os.path.join(results_dir,'csv')
json_dir = os.path.join(results_dir,'json')
exp_dir = os.path.abspath(os.path.join(proj_dir,'behavioral_experiments'))
png_dir = os.path.abspath(os.path.join(datavol_dir,'png'))

## add helpers to python path
if os.path.join(proj_dir,'stimuli') not in sys.path:
    sys.path.append(os.path.join(proj_dir,'stimuli'))
    
if not os.path.exists(results_dir):
    os.makedirs(results_dir)
    
if not os.path.exists(plot_dir):
    os.makedirs(plot_dir)   
    
if not os.path.exists(csv_dir):
    os.makedirs(csv_dir)       

## Read dataframes

In [4]:
# read in dataframes from each eventType
df_block = pd.read_csv('../../results/csv/df_block_cogsci21.csv')
df_chat = pd.read_csv('../../results/csv/df_chat_cogsci21.csv')
df_exit = pd.read_csv('../../results/csv/df_exit_cogsci21.csv')
df_trial = pd.read_csv('../../results/csv/df_trial_cogsci21.csv')

In [5]:
print('n:', df_block.gameid.nunique())

n: 73


In [6]:
# iterationNames
list(df_trial.iterationName.unique())

['pilot0', 'pilot1', 'pilot2', 'pilot3', 'pilot4', 'pilot4b']

## Exclusion criteria

In [7]:
# 75% Accuracy on 75% of trials
df75 = pd.DataFrame(df_trial.groupby(['gameid', 'trialNum'])['trialScore'].sum()>75).groupby(['gameid']).sum()
df75['trials'] = df75['trialScore']

df75 = df75[df75['trials']>=9]
includedGames = list(df75.reset_index().gameid)

print("Total dyads achieving 75% Accuracy on 75% of trials:",len(df75))

Total dyads achieving 75% Accuracy on 75% of trials: 49


In [8]:
# Exclude from analysis
df_block = df_block[df_block.gameid.isin(includedGames)]
df_chat = df_chat[df_chat.gameid.isin(includedGames)]
df_exit = df_exit[df_exit.gameid.isin(includedGames)]
df_trial = df_trial[df_trial.gameid.isin(includedGames)]

### Basic linguistic processing

In [9]:
#create columns for char and word counts
df_chat['word_count'] = df_chat['content'].str.split(' ').str.len()
df_chat['char_count'] = df_chat['content'].str.len()
df_chat["timeElapsedInTurn"] = pd.to_numeric(df_chat['timeElapsedInTurn'])

# add to trial df
trial_sums = df_chat[['gameid','trialNum','word_count','char_count']].groupby(['gameid','trialNum']).sum().reset_index()
df_trial = df_trial.merge(trial_sums, how='outer',on=['gameid','trialNum'])

# message countsa
counts = df_chat.groupby(['gameid','trialNum'])[['iterationName']].count().reset_index()\
    .rename(columns={'iterationName':'n_messages'})
df_trial = df_trial.merge(counts, how='left', on=['gameid','trialNum'])

## Task performance

For accuracy, see .Rmd